# Auditoria de Dataset Pre-Entrenamiento

## Objetivo
Evaluar si el dataset es confiable y usable antes de entrenar. No se entrena ningun modelo en esta fase.

## Dataset
Ruta sugerida: ../DATASET ORIGINAL/DATASET_FINAL_HACKATHON_2026.parquet


In [1]:
import pandas as pd
from pathlib import Path

dataset_path = Path('..') / 'DATASET ORIGINAL' / 'DATASET_FINAL_HACKATHON_2026.parquet'
df = pd.read_parquet(dataset_path)
df.head()


,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,...,sched_minute_of_day,distance_bin,LATITUDE,LONGITUDE,STATION_KEY,DIST_MET_KM,TEMP,WIND_SPD,PRECIP_1H,CLIMATE_SEVERITY_IDX
0,2020,4,1,3,2020-04-01,9E,ATL,"Atlanta, GA",CSG,"Columbus, GA",...,1202,"(10.999, 228.0]",33.636665,-84.428055,722190-13874,1.488682,15.0,6.2,0.0,0
1,2020,4,1,3,2020-04-01,9E,DTW,"Detroit, MI",EVV,"Evansville, IN",...,1226,"(337.0, 438.0]",42.212502,-83.353333,725370-94847,2.759307,9.4,3.1,0.0,0
2,2020,4,1,3,2020-04-01,9E,ATL,"Atlanta, GA",RST,"Rochester, MN",...,832,"(674.0, 837.0]",33.636665,-84.428055,722190-13874,1.488682,7.2,6.7,0.0,0
3,2020,4,1,3,2020-04-01,9E,RST,"Rochester, MN",ATL,"Atlanta, GA",...,959,"(674.0, 837.0]",43.910000,-92.496666,726440-14925,0.764777,3.3,7.2,0.0,0
4,2020,4,1,3,2020-04-01,9E,DTW,"Detroit, MI",TYS,"Knoxville, TN",...,1220,"(438.0, 562.0]",42.212502,-83.353333,725370-94847,2.759307,9.4,3.1,0.0,0


## 1. Verificar filas, columnas y tipos


In [2]:
df.shape, df.dtypes


((35669175, 25),
 YEAR                             int16
 MONTH                             int8
 DAY_OF_MONTH                      int8
 DAY_OF_WEEK                       int8
 FL_DATE                 datetime64[ns]
 OP_UNIQUE_CARRIER             category
 ORIGIN                        category
 ORIGIN_CITY_NAME              category
 DEST                          category
 DEST_CITY_NAME                category
 CRS_DEP_TIME                     int16
 DEP_DELAY                      float32
 DEP_DEL15                         int8
 DISTANCE                       float32
 DEP_HOUR                          int8
 sched_minute_of_day              int16
 distance_bin                  category
 LATITUDE                       float32
 LONGITUDE                      float32
 STATION_KEY                     object
 DIST_MET_KM                    float32
 TEMP                           float32
 WIND_SPD                       float32
 PRECIP_1H                      float32
 CLIMATE_SEVERITY_IDX  

## 2. Analizar nulos y nulos encubiertos
Nulos encubiertos: strings vacias, 'NA', 'N/A', 'NULL', 'null', 'None', 'nan'.


In [3]:
null_summary = df.isna().mean().sort_values(ascending=False)
null_summary.head(20)


YEAR                   0.0
MONTH                  0.0
DAY_OF_MONTH           0.0
DAY_OF_WEEK            0.0
FL_DATE                0.0
OP_UNIQUE_CARRIER      0.0
ORIGIN                 0.0
ORIGIN_CITY_NAME       0.0
DEST                   0.0
DEST_CITY_NAME         0.0
CRS_DEP_TIME           0.0
DEP_DELAY              0.0
DEP_DEL15              0.0
DISTANCE               0.0
DEP_HOUR               0.0
sched_minute_of_day    0.0
distance_bin           0.0
LATITUDE               0.0
LONGITUDE              0.0
STATION_KEY            0.0
dtype: float64

In [4]:
encubiertos = ['','NA','N/A','NULL','null','None','nan']
obj_cols = df.select_dtypes(include=['object','string']).columns
masked = {}
for col in obj_cols:
    masked[col] = df[col].isin(encubiertos).mean()
pd.Series(masked).sort_values(ascending=False).head(20)


STATION_KEY    0.0
dtype: float64

## 3. Analizar rangos y valores fuera de dominio
Definir rangos esperados por dominio y verificar outliers basicos.


In [5]:
df.describe(include='all').T


,count,unique,top,freq,mean,min,25%,50%,75%,max,std
YEAR,35669175.0,NaN,NaN,NaN,2022.601322,2020.0,2021.0,2023.0,2024.0,2025.0,1.601984
MONTH,35669175.0,NaN,NaN,NaN,6.366319,1.0,3.0,6.0,9.0,12.0,3.371166
DAY_OF_MONTH,35669175.0,NaN,NaN,NaN,15.741241,1.0,8.0,16.0,23.0,31.0,8.7699
DAY_OF_WEEK,35669175.0,NaN,NaN,NaN,3.990993,1.0,2.0,4.0,6.0,7.0,2.005924
FL_DATE,35669175,NaN,NaN,NaN,2023-02-01 21:46:22.546138880,2020-01-01 00:00:00,2021-10-16 00:00:00,2023-03-01 00:00:00,2024-06-20 00:00:00,2025-09-30 00:00:00,NaN
OP_UNIQUE_CARRIER,35669175,18,WN,7035589,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ORIGIN,35669175,384,ATL,1767701,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ORIGIN_CITY_NAME,35669175,377,"Chicago, IL",1841011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DEST,35669175,385,ATL,1767579,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DEST_CITY_NAME,35669175,378,"Chicago, IL",1838919,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 4. Analizar continuidad temporal
Verificar huecos por fecha en la ventana disponible.


In [6]:
# Ajustar columna de fecha si el nombre difiere
fecha_col = None
for c in df.columns:
    if 'fecha' in c.lower() or 'date' in c.lower():
        fecha_col = c
        break
fecha_col


'FL_DATE'

In [7]:
if fecha_col:
    fechas = pd.to_datetime(df[fecha_col], errors='coerce')
    fechas_dt = fechas.dropna().dt.date
    rango = pd.date_range(fechas_dt.min(), fechas_dt.max(), freq='D')
    presentes = pd.Series(fechas_dt.unique())
    faltantes = set(rango.date) - set(presentes)
    len(faltantes)


## 5. Identificar fuga de informacion
Revisar columnas que puedan contener informacion posterior al despegue.


In [8]:
sospechosas = [c for c in df.columns if any(k in c.lower() for k in ['delay','delayed','aterr','arr','dep_time','actual','real'])]
sospechosas


['OP_UNIQUE_CARRIER', 'CRS_DEP_TIME', 'DEP_DELAY']

## 6. Analizar distribucion del target (desbalance)


In [9]:
target_col = 'DEP_DEL15'
df[target_col].value_counts(dropna=False)


DEP_DEL15
0    28934579
1     6734596
Name: count, dtype: int64